<h4>Here is the continuation of exercise2 using some quite amazing tools detecting noise management. Tools such as MAD and Ransac
</h4>

In [1]:
# Noise management
# I intend to use MAD for noise detection
import numpy as np
import pandas as pd

df = pd.read_csv("white_wine_cleaned.csv")

variable = "quality"

# Calculate the median
median = df[variable].median()

# Calculate MAD
mad = np.median(np.abs(df[variable] - median))

# Define a threshold for noise detection
threshold = 2 * mad

# Detect noise
noise = df[np.abs(df[variable] - median) > threshold]

print("Detected noise using MAD:")
noise

Detected noise using MAD:


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
251,8.5,0.260,0.21,16.20,0.074,41.0,197.0,0.99800,3.02,0.50,9.8,3
253,5.8,0.240,0.44,3.50,0.029,5.0,109.0,0.99130,3.53,0.43,11.7,3
294,9.1,0.590,0.38,1.60,0.066,34.0,182.0,0.99680,3.23,0.38,8.5,3
445,7.1,0.320,0.32,11.00,0.038,16.0,66.0,0.99370,3.24,0.40,11.5,3
740,6.9,0.390,0.40,4.60,0.022,5.0,19.0,0.99150,3.31,0.37,12.6,3
774,9.1,0.270,0.45,10.60,0.035,28.0,124.0,0.99700,3.20,0.46,10.4,9
820,6.6,0.360,0.29,1.60,0.021,24.0,85.0,0.98965,3.41,0.61,12.4,9
827,7.4,0.240,0.36,2.00,0.031,27.0,139.0,0.99055,3.28,0.48,12.5,9
873,10.3,0.170,0.47,1.40,0.037,5.0,33.0,0.99390,2.89,0.28,9.6,3
876,6.9,0.360,0.34,4.20,0.018,57.0,119.0,0.98980,3.28,0.36,12.7,9


<h4>Based on the tool Median Absolute Deviation (MAD) used to manage Noise detection in the white wine dataset. Using the variable 'quality', so any value in quality that deviates from the median by more than twice the MAD is flagged as noise. And the tool MAD helps noise to detect variables that values are identified by their distance from the central quality value (median) relative to MAD.</h4>
<h4>And it can be seen that the dataset has noise in almost all the variables. The detected noise values are generally due to their extreme values in the dataset, especially in quality, and high/low values in 'sulphates', 'chlorides', 'free sulfur dioxide', and 'total sulfur dioxide'. The presence of these extreme values deviates significantly from the median (MAD) pattern, which I think the reason the variables are flagged.</h4>